In [1]:
from datetime import datetime
import traceback
import time
import json
import os
import re

from imutils.paths import list_files
import requests

In [2]:
IS_JUPYTER = True
ROOT_PATH  = '/home/jovyan/project' if IS_JUPYTER else '/config/workspace/project'
DATA_PATH  = f'{ROOT_PATH}/DoveNest/informations/jsons'

JSON_PATH        = f'{ROOT_PATH}/utils/configs'
JSON_BACKUP_PATH = f'{ROOT_PATH}/BACKUPS/configs'

In [3]:
URLS = {
              'features' : 'http://store.steampowered.com/api/featured/',
            'categories' : 'http://store.steampowered.com/api/featuredcategories/?l=koreana'
        }

In [4]:
unix2datetime = lambda unixtime: str(datetime.fromtimestamp(unixtime))
load_json = lambda json_path: json.loads(open(json_path, 'r').read())
save_json = lambda data, json_path: json.dump(data, open(json_path, 'w'))

def repair_keys(json_path):
    try: keys = load_json(f'{json_path}/config.json')
        
    except Exception as e:
        print(f'[ERR.K-002] json 파일이 깨져 열 수 없습니다. {e}')
        text = open(f'{JSON_BACKUP_PATH}/config.txt', 'r').read().split('\n')
        keys = {platform : key 
                for platform, key in zip(['youtube', 'steam'], text)}
        
        save_paths = [JSON_PATH, JSON_BACKUP_PATH]
        
        print(save_paths)
        for save_path in save_paths:
            
            print(save_path)
            os.makedirs(save_path, exist_ok = True)
            save_json(keys, f'{save_path}/config.json')
        
    finally: return keys

def return_or_print(response):
    
    if response.status_code == 200: return response.json()
    else: print(f'no response data with code : {response.status_code}')

    
get_api  = lambda url: return_or_print(requests.get(url))

def get_info(appid):
    
    response = requests.get(f'https://store.steampowered.com/api/appdetails?appids={appid}&l=korean')
    return return_or_print(response)

In [5]:
def get_json(appid):
    
    json_path = f'{DATA_PATH}/{appid}/{appid}.json'
    if os.path.isfile(json_path):
        json_data = json.loads(open(json_path, 'r').read())
        
        return json_data

    else:
        print(f'[ERR-J0001] <{appid}> json 파일이 존재하지 않습니다.')
        json_data = get_info(appid)
        return {}

In [6]:
if os.path.isfile(f'{JSON_PATH}/config.json'):
    key = repair_keys(JSON_PATH)['steam']
        
else:
    print(f'[ERR.K-001] json 파일이 존재하지 않아 백업 데이터를 로딩합니다.')
    key = repair_keys(JSON_BACKUP_PATH)['steam']

In [7]:
sales = get_api(URLS['categories'])
sales.keys()

dict_keys(['0', '1', '2', '3', '4', '5', '6', 'specials', 'coming_soon', 'top_sellers', 'new_releases', 'genres', 'trailerslideshow', 'status'])

In [8]:
top_games, top_names = [], []
for game in sales['top_sellers']['items']:
    
    appid = game['id']
    try:
        datas     = get_json(appid)
        genre     = ', '.join([d['description'] for d in datas["genres"]])
        thumbnail = datas['header_image'] 
        
        return_data = [game['name'], genre, thumbnail]
        
        if game['name'] not in top_names: 
            top_games.append(return_data)
            top_names.append(game['name'])
            
        else: print(f'[WRN.D-001] 중복된 데이터 입니다. {game["name"]}')
        
    except Exception as e:         
        print(game['name'], e)
        


[ERR-J0001] <2000950> json 파일이 존재하지 않습니다.
Call of Duty®: Modern Warfare® 'genres'
[ERR-J0001] <1669000> json 파일이 존재하지 않습니다.
Age of Wonders 4 'genres'


In [9]:
features = get_api(URLS['features'])
sample   = features['featured_win'][0]

In [13]:
sales['genres']

{'id': 'cat_genres', 'name': '장르'}